In [1]:
# These packages are pre-installed on Google Colab, but are included here to facilitate running this notebook locally
!pip install --quiet matplotlib
!pip install --quiet scikit-learn
!pip install --quiet numpy
!pip install --quiet scipy
!pip install --quiet pacmap
!pip install --quiet leidenalg
!pip install --quiet sinfo
# snRNA-seq analysis
!pip install --quiet scanpy
!pip install --quiet omnipath
!pip install --quiet decoupler


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [31 lines of output]
      /private/var/folders/4f/8j6cxmqx0l15d85h60f9r5wc0000gn/T/pip-install-0nv3qa95/sinfo_65c6d53fd394418c857d3ba99a230392/versioneer.py:421: SyntaxWarning: invalid escape sequence '\

In [2]:
!git clone https://github.com/EugOT/CN-pr-MDD-snRNA-seq.git
%cd /content/CN-pr-MDD-snRNA-seq/

Cloning into 'CN-pr-MDD-snRNA-seq'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 61 (delta 22), reused 19 (delta 9), pack-reused 14 (from 1)
Receiving objects: 100% (61/61), 70.24 MiB | 14.28 MiB/s, done.
Resolving deltas: 100% (22/22), done.
git-lfs filter-process: git-lfs: command not found
fatal: the remote end hung up unexpectedly
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'

[Errno 2] No such file or directory: '/content/CN-pr-MDD-snRNA-seq/'
/Users/macbook/Documents/GitHub/CN-pr-MDD-snRNA-seq/notebooks


/Users/macbook/Desktop/Ale/mlhw/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [3]:
import os
import random
import pacmap
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import decoupler as dc

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
from anndata.experimental.multi_files import AnnCollection


matplotlib.rcParams["pdf.use14corefonts"] = True
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
matplotlib.rcParams["font.family"] = "sans-serif"
matplotlib.rcParams["font.sans-serif"] = ["Helvetica"]
matplotlib.rcParams["figure.max_open_warning"] = 20000

reseed = 42
random.seed(reseed)
np.random.seed(reseed)

sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)
# sc.settings.figdir = PLOTS_DIR
sc.settings.set_figure_params(
    dpi=180, dpi_save=600, vector_friendly=True, format="pdf", transparent=True
)
sc.settings.autoshow = True
sc.settings.autosave = False

/Users/macbook/Desktop/Ale/mlhw/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
samples_males = pd.read_csv("data/PRJNA602867.tsv", delimiter="\t")
print(samples_males.columns)
samples_males = samples_males[["Run", "Condition", "LibraryName", "BioProject", "Sex", "NTotalCells"]]

samples_females = pd.read_csv("data/PRJNA883411.tsv", delimiter="\t")
samples_females = samples_females[["Run", "Condition", "LibraryName", "BioProject", "Sex", "NTotalCells"]]

FileNotFoundError: [Errno 2] No such file or directory: 'data/PRJNA602867.tsv'

In [ ]:
males = sc.read_h5ad(
    "data/PRJNA602867-whole_dataset-fpr_0.001-clusters.h5ad"
)
males.obs['Run'] = males.obs['orig.ident']
males.obs = pd.merge(samples_males, males.obs, on="Run").set_index("cell_name", drop=False)
males.uns["name"] = "PRJNA602867"
sc.pp.filter_cells(males, min_genes=500)
sc.pp.filter_genes(males, min_cells=10)

females = sc.read_h5ad(
    "data/PRJNA883411-whole_dataset-fpr_0.001-clusters.h5ad"
)
females.obs['Run'] = females.obs['orig.ident']
females.obs = pd.merge(females.obs, samples_females, on="Run", how = "inner").set_index("cell_name", drop=False)
females.uns["name"] = "PRJNA883411"
sc.pp.filter_cells(females, min_genes=500)
sc.pp.filter_genes(females, min_cells=10)

In [ ]:
males.obs = males.obs[[
    'cell_name',
    'background_fraction',
    'droplet_efficiency',
    'doublet_score',
    'nFeature_Diff',
    'nCount_Diff',
    'percent_mito',
    'percent_ribo',
    'percent_mito_ribo',
    'percent_hb',
    'log10GenesPerUMI',
    'k_tree',
    'Run',
    'Condition',
    'BioProject',
    'Sex',
    'n_genes']]

females.obs = females.obs[[
    'cell_name',
    'background_fraction',
    'droplet_efficiency',
    'doublet_score',
    'nFeature_Diff',
    'nCount_Diff',
    'percent_mito',
    'percent_ribo',
    'percent_mito_ribo',
    'percent_hb',
    'log10GenesPerUMI',
    'k_tree',
    'Run',
    'Condition',
    'BioProject',
    'Sex',
    'n_genes']]

In [ ]:
sc.experimental.pp.highly_variable_genes(males, flavor="pearson_residuals", n_top_genes=5000)
sc.experimental.pp.highly_variable_genes(females, flavor="pearson_residuals", n_top_genes=5000)
males = males[:, males.var["highly_variable"]]
females = females[:, females.var["highly_variable"]]
males.obs["k_tree"] = males.obs["Sex"].astype(str) + "_" + males.obs["k_tree"].astype(str)
females.obs["k_tree"] = females.obs["Sex"].astype(str) + "_" + females.obs["k_tree"].astype(str)

In [ ]:
adata = ad.concat([males, females], join="inner")
#keep raw and depth-normalized counts for later
adata.layers["raw"] = adata.X.copy()
adata.layers["sqrt_norm"] = np.sqrt(sc.pp.normalize_total(adata, inplace=False)["X"])
adata.raw = adata
sc.experimental.pp.recipe_pearson_residuals(adata, n_top_genes=2000, batch_key="Run")

In [ ]:
hvgs = adata.var["highly_variable"]
embedding = pacmap.PaCMAP(
    n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0, apply_pca=False
)
adata.obsm["X_pacmap"] = embedding.fit_transform(adata.obsm["X_pca"], init="pca")
n_cells = len(adata)
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=50, method='umap')
sc.tl.umap(adata, method='umap')
sc.tl.leiden(adata)

In [ ]:
#Normalize with count depth scaling and apply log
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)


In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='logreg', key_added="leiden")

In [ ]:
sc.pl.rank_genes_groups(adata, key='leiden')

In [ ]:
sc.pl.umap(
    adata,
    color=["PVALB"],
    wspace=0.5,
    ncols=2,
)

In [ ]:
# Query Omnipath and get PanglaoDB
markers = dc.get_resource("PanglaoDB")
markers

In [ ]:
# Filter by canonical_marker and human
markers = markers[
    (markers["human"] == True) & (markers["canonical_marker"] == True)
]
markers

In [ ]:
# Remove duplicated entries
markers = markers[~markers.duplicated(["cell_type", "genesymbol"])]
markers["genesymbol_hs"] = [i.capitalize() for i in markers["genesymbol"]]
markers

In [ ]:
# Enrichment with Over Representation Analysis
dc.run_ora(
    mat=adata,
    net=markers,
    source="cell_type",
    target="genesymbol",
    min_n=3,
    verbose=False,
    use_raw=True,
)


# Object for visualizing the ORA-results
acts = dc.get_acts(adata, obsm_key="ora_estimate")
acts

In [ ]:
with rc_context({'figure.figsize': (6, 3)}):
    sc.pl.embedding(
        acts,
        basis="X_pacmap",
        color=[
            "Astrocytes",
            "Neuroblasts",
            "Neurons",
            "Pyramidal cells",
            "Oligodendrocytes",
            "Oligodendrocyte progenitor cells",
            "Endothelial cells",
            "Mast cells"
        ],
        ncols=4
        )

In [ ]:
# Annotaiton
mean_enr = dc.summarize_acts(acts, groupby="leiden", min_std=1)
annotation_dict = dc.assign_groups(mean_enr)
annotation_dict

In [ ]:
# Add cell type column based on annotation
adata.obs["ora_celltype"] = [
    annotation_dict[str(clust)] for clust in adata.obs["leiden"]
]

In [ ]:
sc.pl.embedding(
    adata,
    basis="X_pacmap",
    color="ora_celltype",
    title="PaCMAP: {feature}".format(feature="ora_celltype"),
    add_outline=True,
    legend_loc="on data",
    legend_fontsize=12,
    legend_fontoutline=2,
    frameon=False,
    palette="tab20",
)

In [ ]:
sc.pl.umap(
    adata,
    color="ora_celltype",
    title="UMAP: {feature}".format(feature="ora_celltype"),
    add_outline=True,
    legend_loc="on data",
    legend_fontsize=12,
    legend_fontoutline=2,
    frameon=False,
    palette="tab20",
)

In [ ]:
#Investigate general distribution of male and female in cell with celltype==Neurons
neurons = adata[adata.obs["ora_celltype"] == "Neurons"]
neurons.obs.groupby("Sex")["cell_name"].count().reset_index()

In [ ]:

#Filter to only keep cells that express the PVALB gene
gene_index = adata.var_names.get_loc('PVALB')
mask = adata.X[:, gene_index] > 0
only_pvalb_expressing_cells = adata[mask]

#Filter to only keep those PVALB expressing cells that are also of celltype==Neuron
only_ovalb_expressing_neurons = only_pvalb_expressing_cells[only_pvalb_expressing_cells.obs["ora_celltype"] == "Neurons"]


In [ ]:
#Group by condition and sex
cluster_condition_counts = only_ovalb_expressing_neurons.obs.groupby(["Sex",'Condition'])['cell_name'].count().reset_index()
cluster_condition_counts